In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import math
import sympy as sp
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
import numpy as np
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.decomposition import PCA
train_data = pd.read_csv('./traininingdata.txt', sep=";")
test_data = pd.read_csv('./testdata.txt', sep=";")
#replace y
train_data.replace({'y': {"yes":'0', "no":'1'}}, inplace=True) 
test_data.replace({'y': {"yes":'0', "no":'1'}}, inplace=True) 
#replace -1 in pdays with a numberlarger than the duration between dates of the dataset
train_data.replace({'pdays': {-1:999}}, inplace=True) 
test_data.replace({'pdays': {-1:999}}, inplace=True)

### column list
col_list=["age","age_class","job","marital","education","default","balance","housing","loan","contact","day","month","duration","campaign","pdays","previous","poutcome","y"]
col_numeric = ["age","balance","day","duration","campaign","pdays","previous"]
col_class = list(set(col_list) - set(col_numeric))
col_unknown = list(['job','education','contact','poutcome'])
col_known = list(set(col_list) - set(col_unknown)-set(['y']))
col_class_known = list(set(col_class) - set(col_unknown)-set(['y']))

# to generate a column that indicates the age class
def age(df):
    bins = pd.cut(df["age"], [17, 30, 40, 55, 100]) # 划分薪资档次
    df["age_class"] = bins
    df[["age_class"]] = df[["age_class"]].astype("str") # 转化为字符串类型
    df.replace({'age_class': {"(17, 30]":'0', "(30, 40]":'1', "(40, 55]":'2', "(55, 100]":'3'}}, inplace=True) 
    ###month
    #df.replace({'month': {"jan":1, "feb":2, "mar":3, "apr":4,"may":5, "jun":6, "jul":7, "aug":8,"sep":9, "oct":10, "nov":11, "dec":12}}, inplace=True) 
    ####
    return df
train_data = age(train_data)
test_data = age(test_data)
####training set
# one-hot encoding of the columns without unkown
data_complete = pd.get_dummies(train_data[col_known])
data_incomplete = train_data[col_unknown]
n = len(data_complete)
#using DecisionTree to repalce the unkonwn
for i in range(len(col_unknown)):
    index_imcomplete = np.array(train_data[train_data[col_unknown[i]]=='unknown'].index)
    index_complete = np.array(list(set(list(range(n)))-set(index_imcomplete)))
    clf = DecisionTreeClassifier()
    clf.fit(data_complete.iloc[index_complete], pd.DataFrame(data_incomplete.iloc[index_complete,i]))
    data_incomplete.iloc[index_imcomplete,i] = clf.predict(data_complete.iloc[index_imcomplete])[0]
# one-hot encoding of the columns with unkown 
data_incomplete_OH = pd.get_dummies(data_incomplete)
train_data_OH_cpt = pd.concat([data_complete,data_incomplete_OH],axis=1)
####SMOTE
oversample = SMOTE()
col_X = list(set(train_data_OH_cpt.columns)-set('y'))
X, y = oversample.fit_resample(train_data_OH_cpt[col_X], train_data['y'])

train_data_OH_cpt = pd.DataFrame(X)
#######testing set
# one-hot encoding of the columns without unkown
data_complete = pd.get_dummies(test_data[col_known])
data_incomplete = test_data[col_unknown]
n = len(data_complete)
#using DecisionTree to repalce the unkonwn
for i in range(len(col_unknown)):
    index_imcomplete = np.array(test_data[test_data[col_unknown[i]]=='unknown'].index)
    index_complete = np.array(list(set(list(range(n)))-set(index_imcomplete)))
    clf = DecisionTreeClassifier()
    clf.fit(data_complete.iloc[index_complete], pd.DataFrame(data_incomplete.iloc[index_complete,i]))
    data_incomplete.iloc[index_imcomplete,i] = clf.predict(data_complete.iloc[index_imcomplete])[0]
# one-hot encoding of the columns with unkown   
data_incomplete_OH = pd.get_dummies(data_incomplete)
test_data_OH_cpt = pd.concat([data_complete,data_incomplete_OH],axis=1)

#to make the continuous features become normal distribution for standardization
def skewness(df):
    df['balance_log'] = df['balance'].map(lambda x : np.log(x+8020))
    df['pdays_log'] = boxcox1p(df['pdays'], boxcox_normmax(df['pdays'] + 2))
    df.drop(columns=['balance', 'pdays'], inplace=True)
    columns = ['duration','campaign','previous']
    for i in columns:
        df[i+'_log'] = df[i].map(lambda x : x**(1/2))
        df.drop(columns=[i], inplace=True)
    return df

train_data_OH_cpt = skewness(train_data_OH_cpt)
test_data_OH_cpt = skewness(test_data_OH_cpt)

C:\Users\loonger\AppData\Local\Temp\ipykernel_18324\1511255368.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_incomplete.iloc[index_imcomplete,i] = clf.predict(data_complete.iloc[index_imcomplete])[0]
C:\Users\loonger\AppData\Local\Temp\ipykernel_18324\1511255368.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_incomplete.iloc[index_imcomplete,i] = clf.predict(data_complete.iloc[index_imcomplete])[0]
C:\Users\loonger\AppData\Local\Temp\ipykernel_18324\1511255368.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [2]:
###数据集标准化没啥大影响，改峰度偏度没啥大影响，
###年龄分类和drop days,drop month有大影响
####standardization
columns = ['balance_log', 'duration_log', 'campaign_log', 'pdays_log', 'previous_log']
def standardization(df):
    df[columns] = df[columns].apply(lambda x:(x-x.mean())/x.std())
    return df
#### pca and standardization for training set
pca = PCA(n_components=3,whiten=True)
train_data_OH_cpt = standardization(train_data_OH_cpt)
train_pca = pd.DataFrame(pca.fit_transform(train_data_OH_cpt),columns=['c1','c2','c3'])
train_data_OH_cpt = pd.concat([train_data_OH_cpt,train_pca],axis=1)

y = pd.DataFrame(y,columns=['y'])
train_data_OH_cpt.insert(train_data.shape[1], 'y', y['y'])
train_data_OH_cpt=train_data_OH_cpt[['day', 'age', 'age_class_0', 'age_class_1', 'age_class_2',
       'age_class_3', 'default_no', 'default_yes', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'loan_no', 'loan_yes', 'housing_no', 'housing_yes', 'marital_divorced',
       'marital_married', 'marital_single', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student', 'job_technician',
       'job_unemployed', 'education_primary', 'education_secondary',
       'education_tertiary', 'contact_cellular', 'contact_telephone',
       'poutcome_failure', 'poutcome_other', 'poutcome_success', 'balance_log',
       'pdays_log', 'duration_log', 'campaign_log', 'previous_log', 'c1','c2','c3', 'y']]
#### pca and standardization for testing set
test_data_OH_cpt = standardization(test_data_OH_cpt)
test_pca = pd.DataFrame(pca.fit_transform(test_data_OH_cpt),columns=['c1','c2','c3'])
test_data_OH_cpt = pd.concat([test_data_OH_cpt,test_pca],axis=1)

test_data_OH_cpt.insert(test_data.shape[1], 'y', test_data['y'])
test_data_OH_cpt=test_data_OH_cpt[['day', 'age', 'age_class_0', 'age_class_1', 'age_class_2',
       'age_class_3', 'default_no', 'default_yes', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'loan_no', 'loan_yes', 'housing_no', 'housing_yes', 'marital_divorced',
       'marital_married', 'marital_single', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student', 'job_technician',
       'job_unemployed', 'education_primary', 'education_secondary',
       'education_tertiary', 'contact_cellular', 'contact_telephone',
       'poutcome_failure', 'poutcome_other', 'poutcome_success', 'balance_log',
       'pdays_log', 'duration_log', 'campaign_log', 'previous_log','c1','c2','c3', 'y']]
train_data_OH_cpt['y'] = train_data_OH_cpt['y'].astype('int')
test_data_OH_cpt['y'] = test_data_OH_cpt['y'].astype('int')


In [3]:
# prediction
col_X = list(set(train_data_OH_cpt.columns)-set(['y']))
lightGBM = lgb.LGBMClassifier(n_estimators = 500,max_depth=3,min_child_samples=10)
lightGBM.fit(train_data_OH_cpt[col_X], train_data_OH_cpt['y'])
predict_y =lightGBM.predict(test_data_OH_cpt[col_X])
accuracy_score(test_data_OH_cpt['y'], predict_y)

0.8546942386376203